In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
adult_census = pd.read_csv("..//M1.03/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(data, target, train_size=0.2, random_state=42)

In [2]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

categorical_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

column_processor = make_column_transformer((categorical_encoder, selector(dtype_include=object)),remainder="passthrough",)

ml_pipeline = Pipeline(
    [
        ("preprocessing", column_processor),
        ("gb_classifier", HistGradientBoostingClassifier(random_state=42)),
    ]
)